In [1]:
'''import sys
sys.path.insert(0, 'nice')
'''
#os.environ['OMP_NUM_THREADS'] = '24'

import numpy as np
import ase.io as ase_io
from ase import Atoms
import tqdm
import time
import copy
from sklearn.linear_model import BayesianRidge
from pathos.multiprocessing import ProcessingPool as Pool
import nice
from nice.transformers import *
from nice.parallelized import *

from matplotlib import pyplot as plt


In [2]:
def process_structures(structures):
    for structure in structures: 
        structure.cell =[120, 120, 120]
        structure.positions += np.asarray((60,60,60))
        structure.pbc=True
        structure.wrap()

In [3]:
structures_train = ase_io.read('structures.xyz', 
                         index = '0:5000')

process_structures(structures_train)

'''structures_val = ase_io.read('structures.xyz', 
                         index = '100:200')'''

#process_structures(structures_val)


"structures_val = ase_io.read('structures.xyz', \n                         index = '100:200')"

In [4]:
energies = np.load('energies.npy')
energies = energies - np.mean(energies)
energies = energies / np.sqrt(np.mean(energies * energies))
energies_train = energies[0:1000]
energies_val = energies[1000:2000]
print(energies_train.shape)
print(energies_val.shape)

(1000,)
(1000,)


In [5]:
HYPERS = {
'interaction_cutoff': 6.3,
'max_radial': 5,
'max_angular': 5,
'gaussian_sigma_type': 'Constant',
'gaussian_sigma_constant': 0.05,
'cutoff_smooth_width': 0.3,
'radial_basis': 'GTO',
}
    

In [6]:
p = Pool(40)

In [7]:
print(len(structures_train))

5000


In [8]:
coefficients = get_rascal_coefficients_parallelized(p, structures_train, HYPERS, 2)
print(coefficients.shape)
coefficients = coefficients.reshape([len(structures_train), 5, coefficients.shape[1], 
                                     coefficients.shape[2], coefficients.shape[3]])

print(coefficients.shape)

100%|██████████| 50/50 [00:01<00:00, 26.53it/s]


(25000, 10, 6, 11)
(5000, 5, 10, 6, 11)


In [9]:
'''def concatenate_data(datas):
    covariants = [datas[i].covariants_ for i in range(len(datas))]
    covariants = np.concatenate(covariants, axis = 0)
    return Data(covariants, datas[0].actual_sizes_, datas[0].importances_, datas[0].raw_importances_)

def transform_parallelized(transformer, *args, task_size = 100):
    print(len(args))
    tasks = []
    for i in range(0, args[0].shape[0], task_size):
        now = []
        for j in range(len(args)):
            now.append(args[j][i : i + task_size])
        tasks.append(now)
    
    def func(task):
        return transformer.transform(*task)
    
    result = [res for res in tqdm.tqdm(p.imap(func, tasks), total = len(tasks))]
    ans = []
    for i in range(len(result[0])):
        now = [result[j][i] for j in range(len(result))]
        ans.append(concatenate_data(now))
        
     print(len(ans))
    print(ans[0].shape)
    return ans'''

'def concatenate_data(datas):\n    covariants = [datas[i].covariants_ for i in range(len(datas))]\n    covariants = np.concatenate(covariants, axis = 0)\n    return Data(covariants, datas[0].actual_sizes_, datas[0].importances_, datas[0].raw_importances_)\n\ndef transform_parallelized(transformer, *args, task_size = 100):\n    print(len(args))\n    tasks = []\n    for i in range(0, args[0].shape[0], task_size):\n        now = []\n        for j in range(len(args)):\n            now.append(args[j][i : i + task_size])\n        tasks.append(now)\n    \n    def func(task):\n        return transformer.transform(*task)\n    \n    result = [res for res in tqdm.tqdm(p.imap(func, tasks), total = len(tasks))]\n    ans = []\n    for i in range(len(result[0])):\n        now = [result[j][i] for j in range(len(result))]\n        ans.append(concatenate_data(now))\n        \n     print(len(ans))\n    print(ans[0].shape)\n    return ans'

In [10]:
def split_coefficients(coefficients):
    c_coefficients = coefficients[:, 0]
    h_coefficients = coefficients[:, 1:]
   
    h_coefficients = np.reshape(h_coefficients, [coefficients.shape[0] * 4, 
                                              coefficients.shape[2], coefficients.shape[3],
                                              coefficients.shape[4]])
    return c_coefficients, h_coefficients

In [11]:
c_coefficients, h_coefficients = split_coefficients(coefficients)
print(c_coefficients.shape)

(5000, 10, 6, 11)


In [12]:
'''begin = time.time()
pst = BSTransformer()
pst.fit(c_coefficients)
print(time.time() - begin)'''

'begin = time.time()\npst = BSTransformer()\npst.fit(c_coefficients)\nprint(time.time() - begin)'

In [13]:
'''begin = time.time()
res = pst.transform(c_coefficients)
print(res[2].shape)
print(time.time() - begin)'''

'begin = time.time()\nres = pst.transform(c_coefficients)\nprint(res[2].shape)\nprint(time.time() - begin)'

In [14]:
def fit(coefficients, num_to_fit):
    c_coefficients, h_coefficients = split_coefficients(coefficients)
    print(c_coefficients.shape)
    begin = time.time()
    c_trans = StandardSequence([StandardBlock(ThresholdExpansioner(),
                                                  IndividualLambdaPCAsBoth(),
                                                  None,
                                                  None),
                                StandardBlock(ThresholdExpansioner(num_expand = 5000),
                                                  IndividualLambdaPCAsBoth(2000),
                                                  ThresholdExpansioner(num_expand = 50000, mode = 'invariants'),
                                                  None),
                                StandardBlock(None,
                                                  None,
                                                  ThresholdExpansioner(num_expand = 50000, mode = 'invariants'),
                                                  None)
                                   ])

    c_trans.fit(c_coefficients[:num_to_fit])
    print(time.time() - begin)
    
    begin = time.time()
    print(h_coefficients[:num_to_fit].shape)
    c_trans = StandardSequence([StandardBlock(ThresholdExpansioner(),
                                                  IndividualLambdaPCAsBoth(),
                                                  None,
                                                  None),
                                StandardBlock(ThresholdExpansioner(num_expand = 5000),
                                                  IndividualLambdaPCAsBoth(2000),
                                                  ThresholdExpansioner(num_expand = 50000, mode = 'invariants'),
                                                  None),
                                StandardBlock(None,
                                                  None,
                                                  ThresholdExpansioner(num_expand = 50000, mode = 'invariants'),
                                                  None)
                                   ])
    h_trans.fit(h_coefficients[:num_to_fit])
    print(time.time() - begin)
    
    return c_trans, h_trans



In [15]:
c_coefficients, h_coefficients = split_coefficients(coefficients)


In [ ]:
c_trans, h_trans = fit(coefficients, 5000)

(5000, 10, 6, 11)


In [ ]:
import pickle
with open("nu_4_data/transformers", "wb") as f:
    pickle.dump([c_trans, h_trans], f, protocol = 4)

In [ ]:
def transform_full(transformer, coefficients, task_size = 100):
    tasks = [coefficients[i : i + task_size]
             for i in range(0, coefficients.shape[0], task_size)]  
    print(len(tasks))
    def wrapped(task):
        return transformer.transform(task, return_only_invariants = True)
    result = [res for res in tqdm.tqdm(p.imap(wrapped, tasks), total = len(tasks))]
    ans = []
    for i in range(len(result[0])):
        now = [result[j][i] for j in range(len(result))]
        ans.append(np.concatenate(now, axis = 0))
        
    return ans

In [ ]:
result = c_trans.transform(c_coefficients, return_only_invariants = True)
for el in result:
    print(el.shape)

In [ ]:
c_coefficients, h_coefficients = split_coefficients(coefficients)
#result = np.concatenate(transform_full(c_trans, c_coefficients), axis = 1)
result = transform_full(c_trans, c_coefficients)
'''print(result[0].shape)
print(result[1].shape)
print(result[2].shape)'''
#print(result.shape)

In [ ]:
with open("nu_4_data/zpbsts_c", "wb") as f:
    pickle.dump(result, f, protocol = 4)

In [ ]:
'''import pickle
with open("nu_2_data/transformers", "rb") as f:
    c_trans, h_trans = pickle.load(f)'''

In [ ]:
result = transform_full(h_trans, h_coefficients)
#print(result.shape)

In [ ]:
with open("nu_4_data/zpbsts_h", "wb") as f:
    pickle.dump(result, f, protocol = 4)

In [ ]:
for el in result:
    print(el.shape)

In [ ]:
class TSTransformer():
    def __init__(self):
        self.initial_ = InitialTransformer()
        self.pca_0_ = IndividualLambdaPCAsBoth()
        self.expansioner_1_ = ThresholdExpansioner()
        self.pca_1_ = IndividualLambdaPCAsBoth()
        self.expansioner_2_invariants_ = ThresholdExpansioner(num_expand = 100000, mode = 'invariants')
        self.pca_2_ = IndividualLambdaPCAsBoth(2000)
        self.expansioner_2_ = ThresholdExpansioner(num_expand = 5000)
        self.expansioner_3_ = ThresholdExpansioner(num_expand = 100000, mode = 'invariants')
        '''self.pca_1_ = IndividualLambdaPCAsBoth(10)
        self.expansioner_2_invariants_ = ThresholdExpansioner(num_expand = 10, mode = 'invariants')
        self.pca_2_ = IndividualLambdaPCAsBoth(10)
        self.expansioner_2_ = ThresholdExpansioner(num_expand = 10)
        self.expansioner_3_ = ThresholdExpansioner(num_expand = 10, mode = 'invariants')'''
        
        '''self.pca_1_ = IndividualLambdaPCAsBoth(1000)
        self.expansioner_2_invariants_ = ThresholdExpansioner(num_expand = 20000, mode = 'invariants')
        self.pca_2_ = IndividualLambdaPCAsBoth(1000)
        self.expansioner_2_ = ThresholdExpansioner(num_expand = 4000)
        self.expansioner_3_ = ThresholdExpansioner(num_expand = 20000, mode = 'invariants')'''
        
        
        
    def fit(self, coefficients):
        print("0")
        data_even_0, data_odd_0 = self.initial_.transform(coefficients)
        print("1")
        self.pca_0_.fit(data_even_0, data_odd_0)
        print("2")
        data_even_0, data_odd_0 = self.pca_0_.transform(data_even_0, data_odd_0, method = 'parallel')
        #print(data_even_0.covariants_.shape[0])
        print("3")
        self.expansioner_1_.fit(data_even_0, data_odd_0, data_even_0, data_odd_0)
        print("4")
        data_even_1, data_odd_1 = self.expansioner_1_.transform(data_even_0, data_odd_0, data_even_0, data_odd_0)
        print("5")
        self.pca_1_.fit(data_even_1, data_odd_1)
        print("6")
        data_even_1, data_odd_1 = self.pca_1_.transform(data_even_1, data_odd_1, method = 'parallel')
        print("7")
        self.expansioner_2_invariants_.fit(data_even_1, data_odd_1, data_even_0, data_odd_0)
        print("8")
        self.expansioner_2_.fit(data_even_1, data_odd_1, data_even_0, data_odd_0)
        print("9")
        data_even_2, data_odd_2 = self.expansioner_2_.transform(data_even_1, data_odd_1, data_even_0, data_odd_0)
        print("10")
        self.pca_2_.fit(data_even_2, data_odd_2)
        print("11")
        data_even_2, data_odd_2 = self.pca_2_.transform(data_even_2, data_odd_2, method = 'parallel')
        print("12")
        self.expansioner_3_.fit(data_even_2, data_odd_2, data_even_0, data_odd_0)
        
    def transform(self, coefficients):                
        data_even_0, data_odd_0 = self.initial_.transform(coefficients)
        data_even_0_t, data_odd_0_t = self.pca_0_.transform(data_even_0, data_odd_0)
        data_even_1, data_odd_1 = self.expansioner_1_.transform(data_even_0_t, data_odd_0_t,
                                                                data_even_0_t, data_odd_0_t)
        data_even_1_t, data_odd_1_t = self.pca_1_.transform(data_even_1, data_odd_1)
        invariants_even_2, _ = self.expansioner_2_invariants_.transform(data_even_1_t, data_odd_1_t, 
                                                             data_even_0_t, data_odd_0_t)
        
        data_even_2, data_odd_2 = self.expansioner_2_.transform(data_even_1, data_odd_1, data_even_0, data_odd_0)
        data_even_2_t, data_odd_2_t = self.pca_2_.transform(data_even_2, data_odd_2)
        
        invariants_even_3, _ = self.expansioner_3_.transform(data_even_2_t, data_odd_2_t, 
                                                             data_even_0_t, data_odd_0_t)
        
        return [data_even_0.covariants_[:, :data_even_0.actual_sizes_[0], 0, 0], 
                data_even_1.covariants_[:, :data_even_1.actual_sizes_[0], 0, 0],
                invariants_even_2, invariants_even_3]
        